In [19]:
import pandas as pd
from preprocess import prepare_for_quantile_transformer
import torch
from model import QuantileEncoderDecoderTransformer, train_one_epoch, evaluate
from model import build_cfg_from_prepared

In [20]:
df = pd.read_csv("raw_data/raw_data_merged.csv")

prepared = prepare_for_quantile_transformer(
    df=df,
    target_col="zuctovacia cena za odchylku",
    L=96*7,     # napr. 7 dní histórie (7*96 = 672)
    H=96,       # 1 deň dopredu
)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
cfg = build_cfg_from_prepared(
    prepared,
    horizon=96,
    use_known_future=True,
    crossing_penalty_weight=0.0,
)

In [25]:
model = QuantileEncoderDecoderTransformer(cfg).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=3e-4)

for epoch in range(10):
    tr = train_one_epoch(model, prepared.train_loader, opt, device)
    va = evaluate(model, prepared.val_loader, device)
    print(epoch, tr, va)

C:\Users\Matúš\Desktop\skola\uniza\phd\.venv\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


KeyboardInterrupt: 